In [307]:
import redis

In [308]:
r = redis.Redis(host='localhost', port=6379, db=0)

In [309]:
r.flushdb()

True

In [310]:
from ipywidgets import HTML
from IPython.display import display#, HTML

def show_redis_data(r):
    html = HTML()
    out = '<table border="1">'
    for key in r.keys():
        t = r.type(key).decode()
#         print(key.decode(), t)
        
        if t == 'string':
            data = r.get(key)
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">string</td>'\
                   f'<td style="padding:3px;">{data.decode()}</td></tr>'
        elif t == 'list':
            l = r.llen(key)
            data = r.lrange(key, 0, l)
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">list</td>'\
                   f'<td style="padding:3px;">[{", ".join([d.decode() for d in data])}]</td></tr>'
        elif t == 'hash':
            data = []
            for key2 in r.hkeys(key):
                data.append(f'{key2.decode()}: {r.hget(key, key2).decode()}')
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">hash</td>'\
                   f'<td style="padding:3px;">{{{", ".join(data)}}}</td></tr>'
        elif t == 'set':
            data = r.smembers(key)
            out += f'<tr><td style="padding:3px;width:150px;">{key.decode()}</td>'\
                   f'<td style="padding:3px;width:50px;">set</td>'\
                   f'<td style="padding:3px;">({", ".join([d.decode() for d in data])})</td></tr>'
        
    out += '</table>'
    
    html.value = out
    display(html)

In [311]:
def add_participants(participants):
    for _id, desc in participants.items():
        r.sadd('participants_ids', _id)
        r.hset(_id, mapping=desc)

In [312]:
participants = {
    'participant_0': {'team': 'google', 'model_name': 'bert'},
    'participant_1': {'team': 'hse', 'model_name': 'hsert'},
    'participant_2': {'team': 'sber', 'model_name': 'sbert'},
    'participant_3': {'team': 'incognito', 'model_name': 'super_rules'}
}

In [313]:
add_participants(participants)

In [314]:
show_redis_data(r)

HTML(value='<table border="1"><tr><td style="padding:3px;width:150px;">participant_0</td><td style="padding:3p…

In [315]:
scores = {
    'participant_0': 96,
    'participant_1': 97,
    'participant_2': 93
}

In [316]:
def add_scores(scores):
    r.zadd('leaderboard', scores)

In [317]:
add_scores(scores)

In [322]:
r.zrange('leaderboard', 0, -1, withscores=True)

[(b'participant_3', 65.0),
 (b'participant_2', 93.0),
 (b'participant_0', 96.0),
 (b'participant_1', 97.0)]

In [319]:
add_scores({'participant_3': 51})

In [321]:
add_scores({'participant_3': 65})

In [327]:
r.zrange('leaderboard', 0, -1, withscores=True)

[(b'participant_3', 65.0), (b'participant_0', 96.0), (b'participant_1', 97.0)]

In [324]:
r.zrevrangebyscore('leaderboard', 100, 90, withscores=True)

[(b'participant_1', 97.0), (b'participant_0', 96.0), (b'participant_2', 93.0)]

In [325]:
def remove_participants(participants):
    for i in participants:
        r.delete(i)
        r.zrem('leaderboard', i)

In [326]:
remove_participants(['participant_2'])

In [328]:
def compute_average_score():
    scores = dict(r.zrange('leaderboard', 0, -1, withscores=True)).values()
    n_scores = r.zcard('leaderboard')
    average = sum([float(s) for s in scores]) / n_scores
    return average

In [329]:
compute_average_score()

86.0

In [299]:
r.zrange('leaderboard', 0, -1)

[b'participant_3', b'participant_0', b'participant_1']

In [330]:
add_participants({'participant_4': {'team': 'mit', 'model_name': 'mit_rules'}})

In [331]:
rules_scores = {
    'participant_4': 35,
    'participant_3': 70,
    'participant_2': 42 
}

In [332]:
r.zadd('rules_leaderboard', rules_scores)

3

In [333]:
r.zrange('rules_leaderboard', 0, -1)

[b'participant_4', b'participant_2', b'participant_3']

In [334]:
r.zinterstore('leaderboard_best', ['leaderboard', 'rules_leaderboard'], aggregate='MAX')

1

In [335]:
r.zrange('leaderboard_best', 0, -1, withscores=True)

[(b'participant_3', 70.0)]